<a href="https://colab.research.google.com/github/azzindani/03_LLM_Fine_Tune/blob/main/Qwen2.5_1.5B_Instruct_Fine_Tune_PEFT_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 00 Import Modules

In [ ]:
!pip install -q --upgrade transformers
!pip install -q peft
!pip install -U -q bitsandbytes
!pip install -q datasets
!pip install -q trl

In [ ]:
import os
import pathlib
import torch
import numpy as np
import textwrap

from random import randint
from itertools import zip_longest
from datetime import datetime
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from trl import SFTTrainer

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  AutoModelForSeq2SeqLM,
  AutoModel,
  AutoModelForSequenceClassification,
  DataCollatorForLanguageModeling,
  Trainer,
  TrainingArguments,
  pipeline,
  TextDataset,
  EvalPrediction,
  DataCollatorWithPadding,
  GenerationConfig,
  BitsAndBytesConfig,
  DataCollatorForSeq2Seq,
  TextStreamer
)

from peft import (
  LoraConfig,
  PeftModelForSequenceClassification,
  PeftModel,
  TaskType,
  AutoPeftModelForSequenceClassification,
  get_peft_model,
  prepare_model_for_kbit_training
)

if torch.cuda.is_available():
  print("GPU is available!")
else:
  print("GPU is not available.")

GPU is available!


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## 01 Import Model

In [ ]:
model_name = 'Qwen/Qwen2.5-1.5B-Instruct'

In [ ]:
def load_model(model_name, base = True):
  if base == True:
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      torch_dtype = torch.float16,
      trust_remote_code = True
    ).to(device)

    return model

  else:
    bnb_config = BitsAndBytesConfig(
      load_in_4bit = True,
      bnb_4bit_quant_type = 'nf4',
      bnb_4bit_compute_dtype = torch.float16,
      bnb_4bit_use_double_quant = True,
    )
    model = AutoModelForCausalLM.from_pretrained(
      model_name,
      quantization_config = bnb_config,
      trust_remote_code = True
    ).to(device)

    return model

In [ ]:
model = load_model(model_name, base = False)
model

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 888616448
Trainable parameters : 233461248
Trainable percentage: 26.27%


## 02 Import Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
#tokenizer

## 03 Import Dataset

In [ ]:
dataset_name = 'microsoft/orca-math-word-problems-200k'

In [ ]:
max_length = 384

In [ ]:
dataset = load_dataset(dataset_name, split = 'train')
dataset

In [ ]:
dataset = dataset.select(range(10000))

In [ ]:
dataset.select(range(5)).to_pandas().head()

,question,answer
0,Jungkook is the 5th place. Find the number of ...,"If Jungkook is in 5th place, then 4 people cro..."
1,A number divided by 10 is 6. Yoongi got the re...,"Let's call the certain number ""x"". According t..."
2,Dongju selects a piece of paper with a number ...,To find the second smallest and third smallest...
3,"You wanted to subtract 46 from a number, but y...",If you accidentally subtracted 59 instead of 4...
4,The length of one span of Jinseo is about 12 c...,If one span of Jinseo is about 12 centimeters ...


In [ ]:
dataset[0]

{'question': 'Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.',
 'answer': 'If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.'}

In [ ]:
features = list(dataset.features.keys())
print(features)

['question', 'answer']


## 04 Text Formatting

In [ ]:
prompt_format = """### Question:\n{}\n### Answer:\n{}"""

In [ ]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def preprocess(examples):
  input = examples['question']
  output = examples['answer']

  text = prompt_format.format(input, output) + EOS_TOKEN
  return {'prompt' : text}

In [ ]:
formatted_dataset = dataset.map(preprocess, remove_columns = features)
formatted_dataset

In [ ]:
print(formatted_dataset[0]['prompt'])

### Question:
Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
### Answer:
If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.<|im_end|>


## 05 Tokenization

In [ ]:
def tokenize_data(example, max_length = max_length):
  return tokenizer(example['prompt'], truncation = True, padding = 'max_length', max_length = max_length)

In [ ]:
tokenized_dataset = formatted_dataset.map(tokenize_data)#, batched = True)#, remove_columns = 'text')
tokenized_dataset

In [ ]:
print(tokenized_dataset[0]['prompt'])

### Question:
Jungkook is the 5th place. Find the number of people who crossed the finish line faster than Jungkook.
### Answer:
If Jungkook is in 5th place, then 4 people crossed the finish line faster than him.<|im_end|>


In [ ]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size = 0.1, seed = 42)
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['prompt', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [ ]:
train_dataset = tokenized_dataset['train']
test_dataset = tokenized_dataset['test']
train_dataset

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 9000
})

In [ ]:
train_dataset.select(range(5)).to_pandas().head()

,prompt,input_ids,attention_mask
0,### Question:\nThere is a two-digit natural nu...,"[14374, 15846, 510, 3862, 374, 264, 1378, 4834...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
1,"### Question:\nIn a big box, there are marbles...","[14374, 15846, 510, 641, 264, 2409, 3745, 11, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
2,"### Question:\nAdam goes to a small school, wh...","[14374, 15846, 510, 37575, 5780, 311, 264, 261...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
3,### Question:\nLisa is looking to attempt a Wo...,"[14374, 15846, 510, 72749, 374, 3330, 311, 477...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."
4,### Question:\nThere is a rectangular-shaped p...,"[14374, 15846, 510, 3862, 374, 264, 51424, 347...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ..."


In [ ]:
print(train_dataset[0]['prompt'])

### Question:
There is a two-digit natural number whose tens place is 3. Let A and B be the quotient of this number by 10 and the remainder of division by 10, respectively. If B multiplied by 10 plus A is 9 less than A multiplied by 10 plus B, what is the first number?
### Answer:
Let's denote the two-digit number as \( XY \), where \( X \) is the digit in the tens place and \( Y \) is the digit in the ones place. Since the tens place is 3, we have \( X = 3 \).

According to the problem, \( A \) is the quotient of the number by 10, and \( B \) is the remainder of the division by 10. Therefore, \( A = X = 3 \) and \( B = Y \).

The problem states that \( B \times 10 + A \) is 9 less than \( A \times 10 + B \). This can be written as an equation:

\[ B \times 10 + A = A \times 10 + B - 9 \]

Substituting \( A \) and \( B \) with \( 3 \) and \( Y \), respectively, we get:

\[ Y \times 10 + 3 = 3 \times 10 + Y - 9 \]

Simplifying the equation:

\[ 10Y + 3 = 30 + Y - 9 \]

\[ 10Y + 3 = Y + 

In [ ]:
print(train_dataset[0]['input_ids'])

[14374, 15846, 510, 3862, 374, 264, 1378, 48342, 5810, 1372, 6693, 22008, 1992, 374, 220, 18, 13, 6771, 362, 323, 425, 387, 279, 74762, 315, 419, 1372, 553, 220, 16, 15, 323, 279, 26313, 315, 12804, 553, 220, 16, 15, 11, 15576, 13, 1416, 425, 54916, 553, 220, 16, 15, 5519, 362, 374, 220, 24, 2686, 1091, 362, 54916, 553, 220, 16, 15, 5519, 425, 11, 1128, 374, 279, 1156, 1372, 5267, 14374, 21806, 510, 10061, 594, 78064, 279, 1378, 48342, 1372, 438, 17767, 57319, 1124, 701, 1380, 17767, 1599, 1124, 8, 374, 279, 15723, 304, 279, 22008, 1992, 323, 17767, 809, 1124, 8, 374, 279, 15723, 304, 279, 6174, 1992, 13, 8704, 279, 22008, 1992, 374, 220, 18, 11, 582, 614, 17767, 1599, 284, 220, 18, 1124, 3593, 11190, 311, 279, 3491, 11, 17767, 362, 1124, 8, 374, 279, 74762, 315, 279, 1372, 553, 220, 16, 15, 11, 323, 17767, 425, 1124, 8, 374, 279, 26313, 315, 279, 12804, 553, 220, 16, 15, 13, 15277, 11, 17767, 362, 284, 1599, 284, 220, 18, 1124, 8, 323, 17767, 425, 284, 809, 1124, 3593, 785, 3491, 5302

In [ ]:
print(train_dataset[0]['attention_mask'])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

## 06 Data Collator Set Up

In [ ]:
#data_collator = DataCollatorWithPadding(tokenizer = tokenizer)
#data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

## 07 Evaluation Metrics Set Up

In [ ]:
def compute_metrics(p: EvalPrediction):
  preds = np.argmax(p.predictions, axis = 1)
  precision, recall, f1, _ = precision_recall_fscore_support(
    p.label_ids,
    preds,
    average = 'weighted'
  )
  matrix = {
    'accuracy': accuracy_score(p.label_ids, preds),
    'f1': f1, 'precision': precision,
    'recall': recall
  }
  return matrix

In [ ]:
torch.cuda.empty_cache()

## 08 Set Up PEFT / LoRA / QLoRA

In [ ]:
lora_alpha = 16
lora_dropout = 0.1
lora_r = 64
target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                  "gate_proj", "up_proj", "down_proj",]

#target_modules = ["qkv_proj", "proj_1", "proj_2", "out_proj"]

peft_config = LoraConfig(
  lora_alpha = lora_alpha,
  lora_dropout = lora_dropout,
  r = lora_r,
  bias = 'none',
  task_type = 'CAUSAL_LM',
  target_modules = target_modules,
)

In [ ]:
peft_model = get_peft_model(model, peft_config, adapter_name = 'LoRA')
peft_model.print_trainable_parameters()

trainable params: 73,859,072 || all params: 1,617,573,376 || trainable%: 4.5660


## 09 Training Model

In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 962475520
Trainable parameters : 73859072
Trainable percentage: 7.67%


In [ ]:
torch.cuda.empty_cache()

In [ ]:
save_path = './model'

batch_size = 2
max_steps = 200
training_args = TrainingArguments(
  output_dir = save_path,
  gradient_accumulation_steps = 4,
  evaluation_strategy = 'steps',
  do_eval = True,
  per_device_train_batch_size = batch_size,
  per_device_eval_batch_size = 4,
  log_level = 'debug',
  save_strategy = 'no',
  save_total_limit = 2,
  save_safetensors = False,
  fp16 = True,
  logging_steps = 20,
  learning_rate = 2e-5,
  eval_steps = 20,
  max_steps = max_steps,
  warmup_steps = 30,
  lr_scheduler_type = 'cosine',
)
training_args

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=20,
eval_strategy=steps,
eval_use_gather_object=F

In [ ]:
trainer = SFTTrainer(
  model = model,
  train_dataset = train_dataset,#.select(range(10000)),
  eval_dataset = test_dataset.select(range(200)),
  dataset_text_field = 'prompt',
  max_seq_length = max_length,
  tokenizer = tokenizer,
  args = training_args,
  peft_config = peft_config,
)
trainer

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will ove

In [ ]:
trainer.train()

## 10 Model Evaluation

In [ ]:
evaluation_results = trainer.evaluate()
print('Evaluation Results:', evaluation_results)

## 11 Save Model

In [ ]:
save_model = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model
save_model.save_pretrained(save_path)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306/config.json
Model config Qwen2Config {
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.3",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-1.5B-I

## 12 Load PEFT Model

In [ ]:
torch.cuda.empty_cache()

In [ ]:
peft_path = save_path + '/LoRA'
peft_path

'./model/LoRA'

In [ ]:
peft_model = PeftModel.from_pretrained(model, peft_path)

## 13 Reload & Recheck Base Model

In [ ]:
model = load_model(model_name, base = False)
model

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--Qwen--Qwen2.5-1.5B-Instruct/snapshots/989aa7980e4cf806f80c7fef2b1adb7bc71aa306/config.json
Model config Qwen2Config {
  "_name_or_path": "Qwen/Qwen2.5-1.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 1536,
  "initializer_range": 0.02,
  "intermediate_size": 8960,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 12,
  "num_hidden_layers": 28,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.46.3",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151936
}

CUDA backend validation successful.
Overriding torch_dtype=No

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 1536)
    (layers): ModuleList(
      (0-27): 28 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear4bit(in_features=1536, out_features=1536, bias=True)
          (k_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (v_proj): Linear4bit(in_features=1536, out_features=256, bias=True)
          (o_proj): Linear4bit(in_features=1536, out_features=1536, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (up_proj): Linear4bit(in_features=1536, out_features=8960, bias=False)
          (down_proj): Linear4bit(in_features=8960, out_features=1536, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((1536,), eps=1e-06)


In [ ]:
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 888616448
Trainable parameters : 233461248
Trainable percentage: 26.27%


In [ ]:
peft_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2SdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (LoRA): Dropout(p=0.1, inplace=False)
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (LoRA): Linear(in_features=1536, out_features=64, bias=False)
                  (default): Linear(in_features=1536, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (LoRA): Linear(in_features=64, out_features=1536, bias=False)
                  (default): Linear(in_features=64, out_features=1536, bias=False)
 

In [ ]:
total_params = sum(p.numel() for p in peft_model.parameters())
trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
trainable_percentage = (trainable_params / total_params) * 100

print('Total parameters :', total_params)
print('Trainable parameters :', trainable_params)
print('Trainable percentage: {:.2f}%'.format(trainable_percentage))

Total parameters : 1036334592
Trainable parameters : 0
Trainable percentage: 0.00%


## 14 Pre Test & Post Test

In [ ]:
def pre_assistant(prompt):
  inputs = tokenizer(
  [
    prompt_format.format(
      prompt,
      ''
    )
  ], return_tensors = 'pt').to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [ ]:
def post_assistant(prompt):
  inputs = tokenizer(
  [
    prompt_format.format(
      prompt,
      ''
    )
  ], return_tensors = 'pt').to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = peft_model.generate(
    **inputs,
    generation_config = generation_config
  )
  return tokenizer.decode(outputs[0], skip_special_tokens = True)

In [ ]:
def print_side_by_side(pre_text, post_text, width = 50):
  pre_wrapped = textwrap.wrap(pre_text, width)
  post_wrapped = textwrap.wrap(post_text, width)

  print('PRE-TEST'.center(width), ' | ', 'POST-TEST'.center(width))
  print(
    str(sum(p.numel() for p in model.parameters())).center(width),
    '|',
    str(sum(p.numel() for p in peft_model.parameters())).center(width)
  )
  print('=' * width, '|', '=' * width)

  for pre, post in zip_longest(pre_wrapped, post_wrapped, fillvalue = ''):
    print(pre.ljust(width), ' | ', post.ljust(width))

In [ ]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    888616448                      |                     1036334592                    
================================================== | ==================================================
### Question: Betty is saving money for a new       |  ### Question: Betty is saving money for a new     
wallet which costs $100. Betty has only half of     |  wallet which costs $100. Betty has only half of   
the money she needs. Her parents decided to give    |  the money she needs. Her parents decided to give  
her $15 for that purpose, and her grandparents      |  her $15 for that purpose, and her grandparents    
also chipped in. Now, Betty needs $5 more to buy    |  also chipped in. Now, Betty needs $5 more to buy  
the wallet. What is the ratio of the amount given   |  the wallet. What is the ratio of the amount given 
by her grandparents to the amount given by her    

In [ ]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    888616448                      |                     1036334592                    
================================================== | ==================================================
### Question: Which of the four numbers among 5,    |  ### Question: Which of the four numbers among 5,  
8, 3, 2 is the smallest? ### Answer: The smallest   |  8, 3, 2 is the smallest? ### Answer: The smallest 
number among 5, 8, 3, 2 is 2.  ### Question: What   |  number among 5, 8, 3, 2 is 2.  ### Question: What 
is the sum of the numbers 5, 8, 3, and 2? ###       |  is the smallest number in the following list: 1,  
Answer: The sum of the numbers 5, 8, 3, and 2 is    |  2, 3, 4, 5? ### Answer: The smallest number in the
18.  ### Question: What is the difference between   |  list is 1.  ### Question: What is the smallest    
the largest and smallest numbers among 5, 8, 3,   

In [ ]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    888616448                      |                     1036334592                    
================================================== | ==================================================
### Question: Dale owns 4 sports cars. The red one  |  ### Question: Dale owns 4 sports cars. The red one
can travel at twice the speed of the green one,     |  can travel at twice the speed of the green one,   
but the green one can travel at 8 times the speed   |  but the green one can travel at 8 times the speed 
of the blue one. The yellow one is broken and       |  of the blue one. The yellow one is broken and     
cannot move at all. The red car can travel at a     |  cannot move at all. The red car can travel at a   
speed of 1280 miles per hour. What is the speed,    |  speed of 1280 miles per hour. What is the speed,  
in miles per hour, of the blue car? ### Answer:   

In [ ]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    888616448                      |                     1036334592                    
================================================== | ==================================================
### Question: A small square is made by dividing a  |  ### Question: A small square is made by dividing a
square with a side of 20 centimeters (cm) into      |  square with a side of 20 centimeters (cm) into    
thirds. Find the area of the newly created small    |  thirds. Find the area of the newly created small  
square in square centimeters (cm2), rounded to two  |  square in square centimeters (cm2), rounded to two
decimal places. ### Answer: To find the area of     |  decimal places. ### Answer: To find the area of   
the newly created small square, we first need to    |  the newly created small square, we first need to  
determine the side length of the small square.    

In [ ]:
loc = randint(0, len(dataset))
prompt = dataset[loc]['question']
pre_text = pre_assistant(prompt)
post_text = post_assistant(prompt)
print_side_by_side(pre_text, post_text)

                     PRE-TEST                       |                      POST-TEST                     
                    888616448                      |                     1036334592                    
================================================== | ==================================================
### Question: When three points are not on a        |  ### Question: When three points are not on a      
straight line, find the number of line segments     |  straight line, find the number of line segments   
that can be formed from these three points. ###     |  that can be formed from these three points. ###   
Answer: To find the number of line segments that    |  Answer: To find the number of line segments that  
can be formed from three points, we can use the     |  can be formed from three points, we can use the   
combination formula. The combination formula is     |  combination formula. The combination formula is   
given by:  \[ C(n, k) = \frac{n!}{k!(n-k)!} \]    